<a href="https://colab.research.google.com/github/ikanx101/G-Colab/blob/main/Image_comparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Comparison Algorithm

Cara pakai: 
1. _Upload_ `2` _images_ yang hendak dibandingkan.
1. Tuliskan nama _files_ sesuai dengan peruntukkannya:
  - _File_ gambar yang dijadikan basis.
  - _File_ gambar yang akan dibandingkan.
  - Masukkan nilai sensitivitas yang dikehendaki.
1. Silakan run _code chunks_:
  - _Chunk_ 1 cukup di-_run_ sekali saja.
  - Jika ingin menggunakan gambar lain atau mengganti _files_, cukup _run_ _chunk_ 2 dan seterusnya.
1. Tunggu prosesnya hingga selesai. Akan ada pesan `-DONE-` di akhir baris.

_Created by_: [Ikang](https://ikanx101.com/)

_feel free to discuss_

In [ ]:
#@title __Chunk 1__: Cukup _run chunk_ ini sekali di awal Anda membuka _notebook_ ini.
# Install berbagai macam alat dan bumbu
system("apt install libfftw3-dev")
install.packages("BiocManager")
BiocManager::install("EBImage")
install.packages("ggpubr")

In [ ]:
#@title Masukkan nama _files_ lengkap beserta _extention_-nya di sini:

# sucikan hati
rm(list=ls())

# panggil libraries
library(EBImage)
library(dplyr)
library(grid)
library(ggplot2)
library(ggpubr)

nama_1 <- "asli.png" #@param {type:"string"}
nama_2 <- "new.png" #@param {type:"string"}
sensitivitas <- 0.1 #@param {type:"number"}

In [ ]:
#@title Dimensi dari kedua _images_:
# membaca kedua gambar
img_1 = readImage(nama_1)
img_2 = readImage(nama_2)

print("======================")
print("--Dimensi image 1--")
print(paste("width:",dim(img_1)[1]))
print(paste("height:",dim(img_1)[2]))
print("======================")
print("--Dimensi image 2--")
print(paste("width:",dim(img_2)[1]))
print(paste("height:",dim(img_2)[2]))
print("======================")
ket = 
 ifelse(dim(img_1)[1] == dim(img_2)[1] &
        dim(img_1)[2] == dim(img_2)[2],
      "Kedua images memiliki dimensi yang sama",
      "Kedua images memiliki dimensi yang berbeda")
print("Keterangan:")
print(ket)

In [ ]:
#@title Proses _resizing_ dan perhitungan kesamaan antara kedua _images_: 
w = max(dim(img_1)[1],dim(img_2)[1])
h = max(dim(img_1)[2],dim(img_2)[2])
img_1 = resize(img_1,w,h)
img_2 = resize(img_2,w,h)

gbr_1 = imageData(img_1) 
gbr_2 = imageData(img_2)

mat_gam = abs(gbr_1-gbr_2) < sensitivitas
result = which(mat_gam == F, arr.ind = T)
result = as.data.frame(result)
result =
        result %>% 
        group_by(dim1,dim2) %>% 
        summarise(freq = n()) %>% 
        ungroup() %>%
        filter(freq == max(freq))

image_new = readImage(nama_2)
g = rasterGrob(image_new,width=unit(1,"npc"), height=unit(1,"npc"), interpolate=F)
      
qplot(1:w, 1:w, geom="blank") +
      annotation_custom(g, xmin=1, xmax=w, ymin=1, ymax=h) +
      geom_point(aes(x = result$dim1,
                     y = h-result$dim2),
                 size = .05,
                 color = "darkred",
                 shape = 4) +
      theme_minimal() +
      labs(title = "Apa saja perbedaan dari gambar ini ke gambar basis?",
           subtitle = "Algoritma Match Matriks Gambar",
           caption = "Dibuat oleh\nikanx101.com\nmenggunakan R") +
      theme(axis.text = element_blank(),
            axis.title = element_blank())

print("- DONE -")